In [1]:
import numpy as np
import pandas as pd

In [2]:
# from energy_consumption.models.combination.combined_DEMO import get_combined_DEMO
from energy_consumption.models.lasso.lasso import get_Lasso_forecasts
from dax.models.combination.combined_new import combine_models
from energy_consumption.models.quantreg.quantreg import get_QuantReg_forecasts
from energy_consumption.models.XGBoost.XGBoost import get_XGBoost_forecasts

from energy_consumption.models.quantreg.quantreg import get_QuantReg_forecasts


In [ ]:
energyforecasts2 = get_Lasso_forecasts()

In [3]:
energyforecasts = get_QuantReg_forecasts()
energyforecasts

100%|██████████| 89/89 [00:59<00:00,  1.49it/s]


did you update weather?
did you update weather?


,date_time,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-19 12:00:00,2024-01-17,energy,36 hour,58.177284,65.753941,67.686303,69.491146,71.863154
1,2024-01-19 16:00:00,2024-01-17,energy,40 hour,54.879059,61.722938,64.154958,66.023732,68.633461
2,2024-01-19 20:00:00,2024-01-17,energy,44 hour,53.768784,60.792566,62.844492,63.572996,64.787048
3,2024-01-20 12:00:00,2024-01-17,energy,60 hour,50.645761,57.598208,59.616702,62.480074,66.685676
4,2024-01-20 16:00:00,2024-01-17,energy,64 hour,47.347535,53.567206,56.085357,59.012661,63.455983
5,2024-01-20 20:00:00,2024-01-17,energy,68 hour,46.237261,52.636833,54.774891,56.561924,59.609571


In [4]:
if 'date_time' in energyforecasts.columns:
    energyforecasts_m2 = energyforecasts.drop(columns=['date_time'])
energyforecasts_m2

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-17,energy,36 hour,58.177284,65.753941,67.686303,69.491146,71.863154
1,2024-01-17,energy,40 hour,54.879059,61.722938,64.154958,66.023732,68.633461
2,2024-01-17,energy,44 hour,53.768784,60.792566,62.844492,63.572996,64.787048
3,2024-01-17,energy,60 hour,50.645761,57.598208,59.616702,62.480074,66.685676
4,2024-01-17,energy,64 hour,47.347535,53.567206,56.085357,59.012661,63.455983
5,2024-01-17,energy,68 hour,46.237261,52.636833,54.774891,56.561924,59.609571


In [5]:
df_sub_dax = combine_models()
df_sub_dax

c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


update unemployment rate on: https://statistik.arbeitsagentur.de/DE/Navigation/Statistiken/Interaktive-Statistiken/Zeitreihen/Lange-Zeitreihen-Nav.html
1104
958


c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmo

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,,,,
2024-01-11,2024-01-10,DAX,1 day,-1.94371,-0.331706,0.018696,0.492039,1.80371
2024-01-12,2024-01-10,DAX,2 day,-0.867044,-0.167619,0.02954,0.225364,0.815918
2024-01-15,2024-01-10,DAX,5 day,-2.80324,-0.565488,0.172514,0.768345,2.859062
2024-01-16,2024-01-10,DAX,6 day,-4.216667,-0.865925,0.146337,1.238233,3.500745
2024-01-17,2024-01-10,DAX,7 day,-5.760401,-1.975153,-0.214216,1.520983,4.961363


In [6]:
from datetime import datetime
import pandas as pd 

weeks = ['0 week', '1 week', '2 week', '3 week', '4 week']
date_st = (datetime.today().strftime('%Y-%m-%d'))

df_sub_inf = pd.DataFrame({
    "forecast_date": date_st,
    "target": "infections",
    "horizon": weeks,
    "q0.025": np.nan,
    "q0.25": np.nan,
    "q0.5": np.nan,
    "q0.75": np.nan,
    "q0.975": np.nan})

In [7]:
submission_frame = pd.merge(df_sub_dax, energyforecasts_m2, how='outer')
submission_frame = pd.merge(submission_frame, df_sub_inf, how='outer')

In [8]:
submission_frame = submission_frame.drop(columns={'date_time'})

KeyError: "['date_time'] not found in axis"

In [9]:
submission_frame

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-10,DAX,1 day,-1.94371,-0.331706,0.018696,0.492039,1.80371
1,2024-01-10,DAX,2 day,-0.867044,-0.167619,0.02954,0.225364,0.815918
2,2024-01-10,DAX,5 day,-2.80324,-0.565488,0.172514,0.768345,2.859062
3,2024-01-10,DAX,6 day,-4.216667,-0.865925,0.146337,1.238233,3.500745
4,2024-01-10,DAX,7 day,-5.760401,-1.975153,-0.214216,1.520983,4.961363
5,2024-01-17,energy,36 hour,58.177284,65.753941,67.686303,69.491146,71.863154
6,2024-01-17,energy,40 hour,54.879059,61.722938,64.154958,66.023732,68.633461
7,2024-01-17,energy,44 hour,53.768784,60.792566,62.844492,63.572996,64.787048
8,2024-01-17,energy,60 hour,50.645761,57.598208,59.616702,62.480074,66.685676
9,2024-01-17,energy,64 hour,47.347535,53.567206,56.085357,59.012661,63.455983


In [10]:
date_string = date_st
date_obj = datetime.strptime(date_string, "%Y-%m-%d")
formatted_date = date_obj.strftime("%Y%m%d")

submission_frame.to_csv(
    str('C:/Users/Maria/Documents/Studium/Pyhton Projekte/PTSFC/submission_files/' +
        formatted_date + '_BrienneVonTarth'),
    index=False, na_rep="NA")